In [39]:
import json
import numpy as np
import pandas as pd

verbose = True

n = 0 # Кол-во переменных

def parse_input(file_name):
    global n

    with open(file_name, 'r') as f:
        data = json.load(f)

    if any(key not in data for key in ['f', 'goal', 'constraints']):
        raise Exception('incorrect input file structure')

    constraints = data['constraints']
    M = len(constraints) # Кол-во уравнений/неравенств
    n = len(constraints[0]['coefs'])
    A = [[0] * n for _ in range(M)]
    b = []

    # Сразу приводим к канонической форме
    for constraint_idx, constraint in enumerate(constraints):        
        b.append(constraint['b'])
        A[constraint_idx][0:n] = constraint['coefs']
        if constraint['type'] != 'eq':
            new_x_coef = 1 if constraint['type'] == 'lte' else -1
            for i in range(len(A)):
                A[i].append(0 if i != constraint_idx else new_x_coef)

    c = data['f']
    c.extend([0] * (len(A[0]) - n))

    return A, b, c

A, b, c = parse_input('input.json')

print(f'A:\n{np.array(A)}')
print(f'b:\n{b}')
print(f'c:\n{c}')

A:
[[ 1  0  0  1  0]
 [ 1  1  0  0 -1]
 [ 1  1  1  0  0]]
b:
[1, 2, 3]
c:
[1, 2, 3, 0, 0]


In [40]:
def to_tableau(A, b, c):
    xb = [eq + [x] for eq, x in zip(A, b)]
    z = c + [0]
    return np.array(xb + [z])

def tableau_to_df(tableau):
    global n
    columns = [f'x{x}' for x in range(1, n+1)] + [f's{s}' for s in range(1, len(tableau[0])-n)] + ['b']
    df = pd.DataFrame(tableau, columns=columns)
    return df

tableau = to_tableau(A, b, c)
tableau_to_df(tableau)

,x1,x2,x3,s1,s2,b
0,1,0,0,1,0,1
1,1,1,0,0,-1,2
2,1,1,1,0,0,3
3,1,2,3,0,0,0


In [41]:
N = len(tableau[0])-1 # Кол-во переменных в канонической форме

In [42]:
import math

def has_solution(A, b):
    # Проверяем систему на совместимость с мопощью сравнения рангов матрицы системы и расширенной матрицы системы.
    return np.linalg.matrix_rank(A) == np.linalg.matrix_rank(np.column_stack((A, b)))

def can_be_improved(tableau):
    # Если любой из коэффициентов в целевой функции положительный, то существует направление роста целевой функции.
    z = tableau[-1][:-1]
    return any(x > 0 for x in z)

def get_pivot_pos(tableau, basic_var_idxs):
    # Выбираем переменную с самым большим коэффициентом(производной) в целевой функции для внесения в базис
    z = tableau[-1][:-1]
    max_z_coef = -math.inf
    for col, coef in enumerate(z):
        if col not in basic_var_idxs and coef > max_z_coef:
            max_z_coef = coef
            max_z_coef_col = col
    
    # Находим переменную которая будет удалена из базиса, для этого найдем самое ближайшее ограничение в выбранном направлении
    max_ratio = -1
    max_ratio_row = -1
    removed_basic_var_idx = -1
    for row, eq in enumerate(tableau[:-1]):
        const = eq[-1]
        if eq[max_z_coef_col] == 0 or const == 0:
            continue

        ratio = eq[max_z_coef_col] / const
        if ratio > 0 and ratio > max_ratio:
            max_ratio = ratio
            max_ratio_row = row

            for col, val in enumerate(eq[:-1]):
                if val != 0 and col != max_z_coef_col and col in basic_var_idxs:
                    removed_basic_var_idx = col
        
    return (max_ratio_row, max_z_coef_col), removed_basic_var_idx

In [43]:
def pivot_step(tableau, pivot_pos, basic_var_idxs, removed_basic_var_idx):
    new_tableau = np.zeros(tableau.shape)

    i, j = pivot_pos
    pivot_value = tableau[i][j]
    new_tableau[i] = np.array(tableau[i]) / pivot_value # Делаем опорный элемент еденицей

    for eq_idx, eq in enumerate(tableau):
        if eq_idx != i: # Обнуляем все остальные элементы в опорной колонке        
            multiplier = np.array(new_tableau[i]) * eq[j]
            new_tableau[eq_idx] = np.array(eq) - multiplier

    basic_var_idxs.add(j)
    basic_var_idxs.remove(removed_basic_var_idx)

    return new_tableau

In [44]:
def simplex(A, b, c):
    global n, N

    tableau = to_tableau(A, b, c)

    basic_var_idxs = set(range(N-n, N))

    if verbose:
        print(f'start tableau:\n{tableau_to_df(tableau)}\n\n')
    while can_be_improved(tableau):
        pivot_pos, removed_basic_var_idx = get_pivot_pos(tableau, basic_var_idxs)
        tableau = pivot_step(tableau, pivot_pos, basic_var_idxs, removed_basic_var_idx)
        if verbose:
            print(f'pivot = {pivot_pos}, step:\n{tableau_to_df(tableau)}\n\n')
    
    if verbose:
        print(f'end tableau:\n{tableau_to_df(tableau)}\n\n')

    solution = get_solution(tableau)
    if verbose:
        print(f'solution: {solution}')
    return solution

def get_solution(tableau):
    columns = np.array(tableau).T
    solution = []
    for column in columns[:-1]:
        val = 0
        if is_basic(column):
            one_index = column.tolist().index(1)
            val = columns[-1][one_index]
        solution.append(val)

    # # Выражаем изначальные переменные
    tableau = to_tableau(A, b, c)
    for i in range(n):
        eq = tableau[i]
        div = eq[i]
        eq[i] = 0
        sm = (eq[-1] - (eq[:-1] * solution).sum())
        solution[i] = sm / div

    return solution[:n]

def is_basic(column):
    return sum(column) == 1 and len([c for c in column if c == 0]) == len(column) - 1

In [45]:
simplex(A, b, c)

start tableau:
   x1  x2  x3  s1  s2  b
0   1   0   0   1   0  1
1   1   1   0   0  -1  2
2   1   1   1   0   0  3
3   1   2   3   0   0  0


(1, 1)
pivot = (1, 1), step:
    x1   x2   x3   s1   s2    b
0  1.0  0.0  0.0  1.0  0.0  1.0
1  1.0  1.0  0.0  0.0 -1.0  2.0
2  0.0  0.0  1.0  0.0  1.0  1.0
3 -1.0  0.0  3.0  0.0  2.0 -4.0


(2, 4)
pivot = (2, 4), step:
    x1   x2   x3   s1   s2    b
0  1.0  0.0  0.0  1.0  0.0  1.0
1  1.0  1.0  1.0  0.0  0.0  3.0
2  0.0  0.0  1.0  0.0  1.0  1.0
3 -1.0  0.0  1.0  0.0  0.0 -6.0


(2, 2)
pivot = (2, 2), step:
    x1   x2   x3   s1   s2    b
0  1.0  0.0  0.0  1.0  0.0  1.0
1  1.0  1.0  0.0  0.0 -1.0  2.0
2  0.0  0.0  1.0  0.0  1.0  1.0
3 -1.0  0.0  0.0  0.0 -1.0 -7.0


end tableau:
    x1   x2   x3   s1   s2    b
0  1.0  0.0  0.0  1.0  0.0  1.0
1  1.0  1.0  0.0  0.0 -1.0  2.0
2  0.0  0.0  1.0  0.0  1.0  1.0
3 -1.0  0.0  0.0  0.0 -1.0 -7.0


solution: [0.0, 2.0, 1.0]


[0.0, 2.0, 1.0]